In [2]:
# imports
import json
import re
import requests
import os
import time
import webbrowser

from bs4 import BeautifulSoup
from rich import print

In [3]:
# parameters
# custom headers to mimic browser
headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:48.0) Gecko/20100101 Firefox/48.0",}

# search url, append with url_num to access a specific page of items
astro_ph_url_base = r"https://arxiv.org/search/advanced?advanced=&terms-0-operator=AND&terms-0-term=&terms-0-field=title&classification-physics=y&classification-physics_archives=astro-ph&classification-include_cross_list=include&date-year=&date-filter_by=date_range&date-from_date=2022-07&date-to_date=&date-date_type=submitted_date&abstracts=show&size=200&order=-announced_date_first&start="

In [9]:
# scrape the website for its articles and get a list with title and link
def recursive_page_scrape(base_url: str, url_num: int = 0, interval_timer: int = 10, do_outputs: bool = False,
                          articles_list: list = [], already_failed: bool = False, recursed: bool = False):
    # reset articles list at the start of the function
    if not recursed: articles_list = []

    # the program should always give an output, http responses fail sometimes
    try:
        # make initial request
        request = requests.get(astro_ph_url_base + str(url_num), headers=headers)

        # build base soup
        soup = BeautifulSoup(request.content, "html.parser")

        # get a list of the article results
        articles = soup.find_all("li", {"class": "arxiv-result"})

        # only parse articles if there are any otherwise return the full list
        if len(articles) <= 0: return articles_list

        # iterate over list of articles
        for article in articles:
            # get data from html tag
            title = article.find("p", {"class": "title"}).text.strip()
            pdf_link = [l["href"] for l in article.find_all("a") if "pdf" in l.text]

            # skip article if no pdf links are present
            if len(pdf_link) != 1: continue

            # only include stellar astro articles (optional)
            if "astro-ph.SR" not in str(article): continue

            # extract single link and append results to list
            pdf_link = pdf_link[0]
            articles_list.append((title, pdf_link))

        # recurse and call back with incremented url_num
        if do_outputs: print(f"Page scraped from {url_num} to {url_num + len(articles)} results. Scraping resumes after {interval_timer} second sleep.")
        time.sleep(interval_timer)
        return recursive_page_scrape(base_url=base_url,
                                     url_num=url_num+len(articles),
                                     interval_timer=interval_timer,
                                     do_outputs=do_outputs,
                                     articles_list=articles_list,
                                     already_failed=already_failed,
                                     recursed=True)
    except Exception as e:
        # recall the code with a already_failed flag set to true, if it fails twice kill it
        if do_outputs: print(f"Exiting due to exception {e}.")
        return articles_list

In [10]:
# get the list of articles
articles = recursive_page_scrape(base_url=astro_ph_url_base, do_outputs=False, interval_timer=0)
print(f"Scraped {len(articles)} articles.")

Scraped 2397 articles.

In [11]:
# save the list of articles to a file
# first the list has to be made into a dict, the key will just be the number of the article
saved_articles_dict = [{num: {"title": art[0], "link": art[1]}} for num, art in enumerate(articles)]
print(f"{len(saved_articles_dict)} articles saved")
# print(json.dumps(saved_articles_dict))

# open/make a new file to dump to
with open("sa_articles_list.json", "w+") as writer: json.dump(saved_articles_dict, writer)

2397 articles saved

In [ ]:
# create and save into a folder
article_folder = "Articles"
os.makedirs(article_folder, exist_ok=True)

# download and save all articles
for number, article in enumerate(articles):
    # define a new title
    new_title = re.sub(r"\$\\.*{([^}]*)}\$", r"\g<1>", str(article[0])).replace(" ", "_").replace("$", "").replace("/", "-").replace("\\", "-") + ".txt"

    # download pdf
    # r = requests.get(article[1], stream=True)
    # with open(os.path.join(article_folder, new_title), "wb") as download: download.write(r.content)
    # save as text file with link for sorting
    with open(os.path.join(article_folder, new_title), "w") as writer: writer.write(article[1])

    # report
    print(f"{number}: Downloaded {new_title}")

In [14]:
# apply sorts
gravi_wave_articles = [_ for _ in articles if "gravi" in _[0] and "wave" in _[0]]

for a in gravi_wave_articles:
    webbrowser.open(a)

[
    'Detecting the heterodyning of gravitational waves',
    'The role of stellar expansion on the formation of gravitational wave sources',
    'Photometric detection of internal gravity waves in upper main-sequence stars. III. Comparison of amplitude 
spectrum fitting and Gaussian process regression using celerite2',
    'Continuous gravitational wave detection to understand the generation mechanism of fast radio bursts',
    'Tidally excited gravity waves in the cores of solar-type stars: resonances and critical-layer formation',
    'Periastron precession effect of $f$-mode dynamical tides on gravitational waves from eccentric double white 
dwarfs',
    'Stochastic excitation of internal gravity waves in rotating late F-type stars: A 3D simulation approach',
    'Opportunistic search for continuous gravitational waves from compact objects in long-period binaries',
    'Panning for gold, but finding helium: discovery of the ultra-stripped supernova SN2019wxt from 
gravitational-wave follow-up observations',
    'Orbital decay of double white dwarfs: beyond gravitational wave radiation effects',
    'Importance of stable mass transfer and stellar winds for the formation of gravitational wave sources',
    'Massive star interiors revealed by gravity wave asteroseismology and high-resolution spectroscopy'
]